## Clean up data on 1st house vote

In [16]:
import pandas as pd

join_field = 'house_dist'

votes = pd.read_csv('original_data/congress_votes_119-2025_h145.csv')
votes['district'] = votes['district'].astype(str)
votes[join_field] = votes['state'] + votes['district'].apply(lambda x: x if len(x) > 1 else '0' + x)
votes = votes[[join_field, 'party', 'vote']]
votes = votes.rename(columns={'vote': 'Vote', 'party': 'Party Name'})
write_path = 'cleaned_data/congress_votes_119-2025_h145_cleaned.csv'
votes.to_csv(write_path, index=False)
print(f"Congressional vote data cleaned and saved to {write_path}")

Congressional vote data cleaned and saved to cleaned_data/congress_votes_119-2025_h145_cleaned.csv


## Clean up shapefile of house districts and convert it to GeoJSON

In [17]:
import geopandas as gpd

# Load the shapefile
house_district_representatives = gpd.read_file('original_data/NTAD_Congressional_Districts_8811650228901827709/Congressional_Districts.shp')

# Verify the projection
print(house_district_representatives.crs)

# Mapping from STATEFP to state abbreviation
statefp_to_abbr = {
    '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT',
    '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL',
    '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD',
    '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE',
    '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND',
    '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD',
    '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV',
    '55': 'WI', '56': 'WY'
}

# Populate the STATE field
house_district_representatives['STATE'] = house_district_representatives['STATEFP'].map(statefp_to_abbr)

# Create the Congressional District field
house_district_representatives[join_field] = house_district_representatives['STATE'] + house_district_representatives['DISTRICT']

# Print the resulting column names
print(house_district_representatives.columns)

columns_to_keep = ['PREFIX', 'FIRSTNAME', 'MIDDLENAME', 'LASTNAME', 'SUFFIX', 'LISTING_NA',
                   'PHONE', 'WEBSITEURL', 'VACANT', 'CONTACTFOR', 'PHOTOURL', 'FACE_BOOK_',
                   'TWITTER_UR', 'YOUTUBE_UR', 'INSTAGRAM_', 'FLICKR_URL', 'PARTY', 'DISTRICT',
                   'STATE', 'VACANCY', 'ROOM_NUM', 'geometry', join_field]

house_district_representatives = house_district_representatives[columns_to_keep]

# Write the cleaned data to a new file
write_path = 'cleaned_data/house_district_representatives_cleaned.shp'
house_district_representatives.to_file(write_path, driver='ESRI Shapefile')
print(f"House district representatives data cleaned and saved to {write_path}")

EPSG:3857
Index(['STATEFP', 'CD119FP', 'GEOID', 'GEOIDFQ', 'NAMELSAD', 'LSAD', 'CDSESSN',
       'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON',
       'OFFICE_ID', 'BIOGUIDE_I', 'OFFICE_AUD', 'PREFIX', 'FIRSTNAME',
       'MIDDLENAME', 'LASTNAME', 'SUFFIX', 'LISTING_NA', 'PHONE', 'WEBSITEURL',
       'VACANT', 'CONTACTFOR', 'PHOTOURL', 'FACE_BOOK_', 'TWITTER_UR',
       'YOUTUBE_UR', 'INSTAGRAM_', 'FLICKR_URL', 'PARTY', 'DISTRICT', 'STATE',
       'VACANCY', 'ROOM_NUM', 'HOB', 'COMMITTEE_', 'LAST_UPDAT', 'geometry',
       'house_dist'],
      dtype='object')
House district representatives data cleaned and saved to cleaned_data/house_district_representatives_cleaned.shp


In [19]:
# Save to PMTTiles format
house_district_representatives.to_file('map_data/house_districts.pmtiles', driver='PMTiles')

## Join and merge all data files

In [20]:

# Join with the cleaned data about how each congressperson voted
house_district_representatives = house_district_representatives.merge(votes, on=join_field, how='left')

# Join with data on medicaid recipients
medicaid_recipients = pd.read_csv('original_data/data-iFF42.csv')

# Rename 'Congressional District' to match the join field
medicaid_recipients.rename(columns={'Congressional District': join_field}, inplace=True)

house_district_representatives = house_district_representatives.merge(medicaid_recipients, on=join_field, how='left')

# Print the resulting column names
print(house_district_representatives.columns)

columns_to_keep = ['PREFIX', 'FIRSTNAME', 'MIDDLENAME', 'LASTNAME', 'SUFFIX', 'LISTING_NA',
                   'PHONE', 'WEBSITEURL', 'VACANT', 'CONTACTFOR', 'PHOTOURL', 'FACE_BOOK_',
                   'TWITTER_UR', 'YOUTUBE_UR', 'INSTAGRAM_', 'FLICKR_URL', 'PARTY', 'DISTRICT',
                   'STATE', 'VACANCY', 'ROOM_NUM', 'geometry', join_field,
                   'Party Name', 'Vote', 'Party', 'Adult', 'Seniors', 'Children',
                   'Individuals with Disabilities', 'ACA Adults', 'Total']

# Filter the GeoDataFrame to keep only the specified columns
house_district_representatives = house_district_representatives[columns_to_keep]


Index(['PREFIX', 'FIRSTNAME', 'MIDDLENAME', 'LASTNAME', 'SUFFIX', 'LISTING_NA',
       'PHONE', 'WEBSITEURL', 'VACANT', 'CONTACTFOR', 'PHOTOURL', 'FACE_BOOK_',
       'TWITTER_UR', 'YOUTUBE_UR', 'INSTAGRAM_', 'FLICKR_URL', 'PARTY',
       'DISTRICT', 'STATE', 'VACANCY', 'ROOM_NUM', 'geometry', 'house_dist',
       'Party Name', 'Vote', 'Party', 'Adult', 'Seniors', 'Children',
       'Individuals with Disabilities', 'ACA Adults', 'Total'],
      dtype='object')
